### Please use this notebook inside google colab environment. In Runtime select "Run All", and after the execution, it will show the accuracy result of NaVA in Toy Dataset.

In [3]:
!pip install coclust
from coclust.evaluation.external import accuracy


In [4]:
%tensorflow_version 1.x
import tensorflow as tf 
import numpy as np
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1UcsXn-hO7APQpDWBcgCwbFbduUePd6e3',
                                    dest_path='./tweets.zip',
                                    unzip=True)

In [5]:

from sklearn.model_selection import train_test_split

train = np.load("tweets.npy",allow_pickle=True)
labels = np.load("labels-tweets.npy",allow_pickle=True)

X_train, X_test, y_train, y_test = train_test_split(train,labels, test_size=0.33, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train,y_train, test_size=0.1, random_state=42)

X_train = [' '.join(x) for x in X_train]
X_test = [' '.join(x) for x in X_test]
X_valid = [' '.join(x) for x in X_valid]


In [6]:
#@title Bag of Words Encode
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_20newsgroups
from random import shuffle

vectorizer = CountVectorizer()

trainX = vectorizer.fit_transform(X_train)
validX = vectorizer.transform(X_valid)
testX = vectorizer.transform(X_test)

voc = vectorizer.vocabulary_


indices = np.arange(trainX.shape[0]) #gets the number of rows 
shuffle(indices)


print('train shape:',trainX.shape,"test shape:",testX.shape)

train shape: (10194, 17915) test shape: (5580, 17915)


In [7]:
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import numpy as np

from tensorflow.keras.utils import to_categorical
t_2D = TSNE(n_components=2).fit_transform(trainX)
kmeans = KMeans(n_clusters=3, random_state=0).fit(t_2D)
import plotly.express as px
fig = px.scatter(x=t_2D[:,0],y=t_2D[:,1],color=kmeans.predict(t_2D))
fig.show()
to_y = to_categorical(kmeans.predict(t_2D))
true = kmeans.predict(t_2D)

In [8]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

code = np.array(y_train)

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(code)
y_test = label_encoder.transform(y_test)

In [9]:
cluster = 3 #size of c
h_len = 400 #30
#0sd  #size of h
batch_size = 32
epochs =  30
eta = 0.00023

In [10]:
ep = 1e-20
temp = 1.0


x = tf.placeholder(tf.float32, shape=[None,trainX.shape[1]])
batch = tf.placeholder(tf.int32, shape=())
y = tf.placeholder(tf.float32, shape=[None,cluster])
L = tf.random.uniform([h_len,trainX.shape[1]],-0.05,0.05)

def encoder_h(x,name="classDecoder"):

        W1 = tf.Variable(tf.random.uniform([trainX.shape[1],1000],0.05,-0.05),name='W1',dtype=tf.float32)
        b1 = tf.Variable(tf.zeros([1000]),name='bias1',dtype=tf.float32)

        layer1 = tf.nn.sigmoid(tf.matmul(x,W1) + b1)

        W2 = tf.Variable(tf.random.uniform([1000,750],0.05,-0.05),name='W2')
        b2 = tf.Variable(tf.zeros([750]),name='bias2',dtype=tf.float32)

        layer2 = tf.nn.sigmoid(tf.matmul(layer1,W2) + b2)

        W3 = tf.Variable(tf.random.uniform([750,h_len],-0.05,0.05),name='WMu')
        b3 = tf.Variable(tf.zeros([h_len]),name='biasMu',dtype=tf.float32)

        W4 = tf.Variable(tf.random.uniform([750,h_len],0,0),name='WSigma')
        b4 = tf.Variable(tf.zeros([h_len]),name='biasSigma',dtype=tf.float32)

        mu = tf.matmul(layer2,W3) + b3
        log_sigma = tf.matmul(layer2,W4) + b4

        return mu,log_sigma

def encoder_c(hs,name="C"):

    with tf.variable_scope(name):
    
        W1 = tf.Variable(tf.random.uniform([trainX.shape[1],1000],-0.05,0.05),name='W1')
        b1 = tf.Variable(tf.zeros([1000]),name='bias1',dtype=tf.float32)

        
        layer1 = tf.nn.tanh(tf.matmul(hs,W1) + b1)

        W2 = tf.Variable(tf.random.uniform([1000,cluster],-0.05,0.05),name='W2')
        b2 = tf.Variable(tf.zeros([cluster]),name='bias2',dtype=tf.float32)

        layer2 = tf.nn.tanh(tf.matmul(layer1,W2) + b2)
        return layer2


def loss(output, y):
    xentropy = tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=y)    
    loss = tf.reduce_mean(xentropy)
    return loss


def decoder(x,h,name='ClassEncoder'):

     with tf.variable_scope(name):

          R = tf.Variable(L,name='RWord')
          b = tf.Variable(tf.zeros([trainX.shape[1]]),name='biasWord')

          h_R = tf.nn.log_softmax((tf.matmul(h,R)-b))

          h_R2 = tf.nn.log_softmax(tf.matmul(h,R))

          un_probs = tf.reduce_sum(tf.multiply(h_R,x),axis=1)

          return un_probs,R,h_R,h_R2,b


#h
eps = tf.random_normal((batch,h_len), 0,1)
mu,log_sigma = encoder_h(x)
h = mu+tf.multiply(tf.exp(log_sigma),eps)

#c 
eps2 = tf.random_uniform((batch,cluster), 0,1)
eps2 = -tf.log(-tf.log(eps2+ep)+ep)
C = encoder_c(x)
C2 = C+eps2
C2 = tf.nn.softmax(C2/temp,axis=1)


# Doing mixture 
logits = []
word_embedding = []
topic = []
topics_wB = []
bias_vector = []

for l in range(0,int(cluster)):

    un_probs,R,h_R,h_R2,bias_vec = decoder(x,h,name="Class_"+str(l))
    
    logits.append(un_probs)
    word_embedding.append(R)
    topic.append(h_R)
    topics_wB.append(h_R2)
    bias_vector.append(bias_vec)

decoder_probs = tf.multiply(logits,tf.transpose(C2))
decoder_probs = tf.reduce_sum(decoder_probs,axis=0)

# Calcule ELBO

kld_c = tf.reduce_sum(tf.nn.softmax(C,axis=1)*tf.log(tf.nn.softmax(C,axis=1)/(1.0/np.float(cluster))),axis=1)
kld_h = -0.5 * tf.reduce_sum(1 - tf.square(mu) + 2 * log_sigma - tf.exp(2 * log_sigma), 1) 

ELBO = decoder_probs - kld_c - kld_h
mean_ELBO = -tf.reduce_mean(ELBO)

optimizer = tf.train.AdamOptimizer(learning_rate=eta)
optimizer2 = tf.train.AdamOptimizer(learning_rate=eta)



clu = loss(C,y)
step = optimizer.minimize(mean_ELBO)
step2 = optimizer.minimize(clu)

sess = tf.Session()
print('Initializing...')

init = tf.global_variables_initializer()
sess.run(init)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Initializing...


In [11]:
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import cosine

keep_elbo_ppr = []

for ll in range(0,10):

    begin = 0
    end = batch_size

    while end <=trainX.shape[0]:

        select = indices[begin:end]

        feed_dict = {x:trainX[select].toarray().reshape(batch_size,trainX.shape[1]),y:to_y[select],batch:batch_size}
        sess.run(step2,feed_dict=feed_dict)
        a = sess.run(clu,feed_dict=feed_dict)
        
        

        begin = end
        end += batch_size
            
    #print ("### EPOCH",ll," #####")
    from sklearn.metrics import accuracy_score
    feed_dict = {x:trainX.toarray(),y:to_y,batch:trainX.shape[1]}
    preds = sess.run(C,feed_dict=feed_dict)
    #print(accuracy_score(true,np.argmax(preds,axis=1)))
    #print(a)

In [12]:
from sklearn.metrics.cluster import *

In [13]:
##@title Train Model and calculate mean(ELBO) and perplexity For Validation Set
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import cosine

keep_elbo_ppr = []

for ll in range(0,epochs):

    begin = 0
    end = batch_size

    while end <=trainX.shape[0]:

        select = indices[begin:end]

        feed_dict = {x:trainX[select].toarray().reshape(batch_size,trainX.shape[1]),batch:batch_size}
        sess.run(step,feed_dict=feed_dict)

        begin = end
        end += batch_size
            
    print ("### EPOCH",ll," #####")
    
    feed_dict = {x:testX.toarray(),batch:testX.shape[0]}
    elbo,c_test,mu_test = sess.run([ELBO,C,mu],feed_dict=feed_dict)
    print('print ELBO mean:',np.mean(elbo))

    # clean memory, importat for large dataset
    feed_dict = {}
    del feed_dict
    feed_dict = 2

    val_matrix = testX.toarray()
    indices_sec = []
    for i,uu in enumerate(np.sum(val_matrix,axis=1)):
        if uu >0:
            indices_sec.append(i)

    indices_sec = np.array(indices_sec)
    val_matrix = testX[indices_sec].toarray()


    feed_dict = {x:val_matrix,batch:val_matrix.shape[0]}
    
    elbo = sess.run(ELBO,feed_dict=feed_dict)

    ppr  = np.exp(-np.sum(elbo/np.sum(val_matrix,axis=1))/np.float(len(indices_sec)))
    print ('Perplexity',ppr)
    print ('ELBO mean:',np.mean(elbo))
    print(homogeneity_score(y_test,np.argmax(c_test,axis=1)),completeness_score(y_test,np.argmax(c_test,axis=1)),v_measure_score(y_test,np.argmax(c_test,axis=1)))
    print(silhouette_score(mu_test,np.argmax(c_test,axis=1)))
    print(davies_bouldin_score(mu_test,np.argmax(c_test,axis=1)))
    print(calinski_harabasz_score(mu_test,np.argmax(c_test,axis=1)))
    print(normalized_mutual_info_score(y_test,np.argmax(c_test,axis=1)))
    print(accuracy(y_test,np.argmax(c_test,axis=1)))
    keep_elbo_ppr.append([np.mean(elbo),ppr])

    # clean memory, importat for large dataset
    del ppr
    del val_matrix

    val_matrix = 0
    ppr = 0

### EPOCH 0  #####
print ELBO mean: -77.51821
Perplexity 14545.100709489454
ELBO mean: -77.64892
0.022806167474488912 0.025277605486130634 0.023978372268868942
-0.12454798
7.307407752759126
63.716120809172956
0.02397837226886894
0.5155913978494624


/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning:

The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.



### EPOCH 1  #####
print ELBO mean: -74.66065
Perplexity 10186.442212113472
ELBO mean: -74.80099
0.016305301556839607 0.01930515227235746 0.017678872103753428
-0.18349288
5.615095439723922
98.75987629568563
0.017678872103753428
0.5173835125448029


/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning:

The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.



### EPOCH 2  #####
print ELBO mean: -73.31475
Perplexity 8577.246500246825
ELBO mean: -73.40889
0.01397236266517726 0.016637502107760726 0.015188908217445325
-0.14596152
5.672805981034288
127.72916786015432
0.015188908217445323
0.5120071684587814


...

/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning:

The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.



### EPOCH 28  #####
print ELBO mean: -68.308075
Perplexity 4581.125067888174
ELBO mean: -68.466
0.06901754556943114 0.05664305374200916 0.062221007447876155
0.16852352
1.746209382817831
1213.485013653791
0.062221007447876155
0.5005376344086021


/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning:

The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.



### EPOCH 29  #####
print ELBO mean: -68.32959
Perplexity 4514.996944186577
ELBO mean: -68.357796
0.07083919541162334 0.05812943648392534 0.0638580474915383
0.19218214
1.6575980289905052
1272.721932163442
0.0638580474915383
0.48333333333333334


/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning:

The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.



In [14]:
feed_dict = {x:trainX.toarray(),batch:trainX.shape[0]}
c_train,h_train,mu_train,test = sess.run([C,h,mu,C2],feed_dict=feed_dict)

# clean memory, importat for large dataset
feed_dict = {}
del feed_dict
feed_dict = 2

feed_dict = {x:testX.toarray(),batch:testX.shape[0]}
c_test,h_test,mu_test,test2 = sess.run([C,h,mu,C2],feed_dict=feed_dict)

# clean memory, importat for large dataset
feed_dict = {}
del feed_dict
feed_dict = 2

feed_dict = {x:testX[0:1].toarray(),batch:1}
words_embedding = sess.run(word_embedding)

feed_dict = {}
del feed_dict
feed_dict = 2


r_train = []
r_train2 = []
cluster = np.argmax(c_train,axis=1)

for sparse,c_cluster,mus in zip(trainX,cluster,mu_train):
    
    s = vectorizer.inverse_transform(sparse.toarray())[0]
    aux = np.zeros(h_len)

    for word in s:
        aux += words_embedding[c_cluster][:,voc[word]]

    if len(s) > 0:
        rs = aux/np.float(len(s))
        rm = (aux+mus)/np.float(len(s)+1)

    r_train.append(rs)
    r_train2.append(rm)

r_test = []
r_test2 = []

cluster = np.argmax(c_test,axis=1)

for sparse,c_cluster,mus in zip(testX,cluster,mu_test):

    s = vectorizer.inverse_transform(sparse.toarray())[0]
    aux = np.zeros(h_len)

    for word in s:
        aux += words_embedding[c_cluster][:,voc[word]]

    if len(s) > 0:

        rs = aux/np.float(len(s))
        rm = (aux+mus)/np.float(len(s)+1)

    r_test.append(rs)
    r_test2.append(rm)

r_train = np.array(r_train)
r_test = np.array(r_test)

r_train2 = np.array(r_train2)
r_test2 = np.array(r_test2)

In [15]:
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import accuracy_score
    

clf = LR().fit(mu_train,y_train);
h_preds = clf.predict(mu_test);

clf = LR().fit(r_train,y_train);
r_preds = clf.predict(r_test);


#clf = LR().fit(r_train2,y_train);
#r_preds2 = clf.predict(r_test2);

print ('Accuracy for H Representation:',accuracy_score(y_test,h_preds))
print ('Accuracy for merge R Representation:',accuracy_score(y_test,r_preds))
#print ('Accuracy for merge R Representation:',accuracy_score(y_test,r_preds2))
 

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



Accuracy for H Representation: 0.7668458781362008
Accuracy for merge R Representation: 0.7625448028673835


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



In [16]:
import numpy as np
from sklearn.manifold import TSNE

h_2D = TSNE(n_components=2).fit_transform(mu_test)
r_2D = TSNE(n_components=2).fit_transform(r_test)
r_2D2 = TSNE(n_components=2).fit_transform(r_test2)


In [20]:
import plotly.express as px
fig = px.scatter(x=h_2D[:,0],y=h_2D[:,1],color=y_test)
fig.show()

In [17]:
import plotly.express as px
fig = px.scatter(x=h_2D[:,0],y=h_2D[:,1],color=np.argmax(c_test,axis=1))
fig.show()

In [18]:
import plotly.express as px
fig = px.scatter(x=r_2D2[:,0],y=r_2D2[:,1],color=y_test)
fig.show()

In [19]:
import plotly.express as px
fig = px.scatter(x=r_2D2[:,0],y=r_2D2[:,1],color=np.argmax(c_test,axis=1))
fig.show()